In [1]:
import pandas as pd
import re
import numpy as np

In [72]:
#Read the excel sheet with the schools ranked for that particular year, downloaded from NECTA Education Dashboard
#It's important to do this for every year because regions and districts change frequently.
excel = pd.read_excel("~/Documents/GitHub/ImportingNECTA/PSLE2017_RANKING.xls", header=2) #tinker with header for correct header

In [73]:
#Review the column names that will be changed
print(excel.columns)

Index(['Color Code: Green: High Performing Schools; Yellow: Medium Performing Schools; Red: Low Performing Schools',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [74]:
def new_columns_dict(data):
    """
    Collect second row for new column headers to be used for .rename() call.
    Assumes new columns are in the second row. Should only be run unless rereading the file later.
    Returns dictionary of {OldColumn: NewColumn}
    """
    #Initialize dict with None values
    columns = {k:None for k in data.columns}
    #Put new column names in indexed
    new_columns = data.iloc[0]
    i = 0
    for column in data.columns:
        columns[column] = new_columns[i]
        i+=1
    data.drop(0, inplace=True)
    return columns

#Renaming the columns
excel.rename(columns=new_columns_dict(excel), inplace=True)

#Check to ensure intended effects. Note: Column names are case sensitive!
print(excel.head())

  CENTRE CODE                     CENTRE NAME DISTRICT NAME REGION NAME  \
1  PS0504-112  ST PETER CLAVER PRIMARY SCHOOL       KARAGWE      KAGERA   
2  PS0501-076      ST SEVERINE PRIMARY SCHOOL    BIHARAMULO      KAGERA   
3  PS1304-001         ALLIANCE PRIMARY SCHOOL   MWANZA JIJI      MWANZA   
4  PS2007-046         SIR JOHN PRIMARY SCHOOL   TANGA MJINI       TANGA   
5  PS1701-062          PALIKAS PRIMARY SCHOOL    KAHAMA MJI   SHINYANGA   

  CLEAN CANDIDATES 2017 NUMBER OF STUDENTS PASSED (A-C)  \
1                    46                              46   
2                    66                              66   
3                    42                              42   
4                    41                              41   
5                    53                              53   

  AVERAGE TOTAL MARKS (/250) 2017 AVERAGE TOTAL MARKS (/250) 2016  \
1                           231.3                          216.55   
2                          227.02                       

In [75]:
#Sanity check for the number of districts we should expect. CENTRE CODE is a unique school code, but it contains
#Region-District identifier PS[two digit region][two digit district]. Compare with Google searches!

print(len(excel["CENTRE CODE"].map(lambda x: x[:x.index("-")]).unique()))

186


In [76]:
#While Region names are guaranteed to be unique, district names may not be (Turns out that they are for those curious).
#In order to ensure unique district names, we make a unique identifier that uses Region-District together. We do so 
#by calling the .groupby() function.

def get_unique_groups(data):
    """
    Uses the pre-processed excel data to return unique Region-District groups with unique CENTRE CODE values.
    """
    groupby = data.groupby(['REGION NAME', 'DISTRICT NAME'])['CENTRE CODE', 'DISTRICT NAME', 'REGION NAME'] #I index for just three columns for clarity

    #Groupby returns an object that has other calling functions. Naturally, we want to check to see if it grouped,
    #so we use .head(). It took a while, but I finally understood what .head is doing with a groupby object. 
    #By not specifying .head(1), I was not getting unique Region-District combinations. Rather it included up to 5 of
    #the Region-District combinations.

    unique_groups = groupby.head(1)

    #Remeber that lambda function used in the sanity check? We are revisiting it here for the next step. We have unique,
    #Region-District combinations in unique_groups, however the CENTRE CODE values are more concise and contain
    #a single value to reflect unique Region-District combinations. We take out school information after the dash
    #using the lambda function for every CENTRE CODE. We get the settingwithcopywarning, but when checking unique_groups,
    #the values did get updated.

    unique_groups.loc[:,"CENTRE CODE"] = unique_groups.loc[:,"CENTRE CODE"].map(lambda x: x[:x.index("-")])
    
    return unique_groups

unique_groups = get_unique_groups(excel)

#Check the values of unique_groups
print(unique_groups.head())


  CENTRE CODE DISTRICT NAME REGION NAME
1      PS0504       KARAGWE      KAGERA
2      PS0501    BIHARAMULO      KAGERA
3      PS1304   MWANZA JIJI      MWANZA
4      PS2007   TANGA MJINI       TANGA
5      PS1701    KAHAMA MJI   SHINYANGA


/Users/Muse/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [78]:
#With unique_groups, we have the identifier and the Region-District it belongs to. Unfortunately, this information was
#not explicitly included when scraping the NECTA site. However, having this reference table/dataframe, allows us
#to now map Districts to each student. Here is an example that uses 2014's data.

#Read the scraped PSLE data.
psle2017 = pd.read_csv("~/...")

def categorize_locations(psle,unique_groups):
    """
    Adds a Region and District column and populates that year's PSLE scores with the locations.
    Expects properly formatted psle and unique_groups value from get_unique_groups function.
    Returns the original dataset with locations attached.
    """

    #Create a new column, District, and set values to NaN. Sets Region values to NaN.
    psle["District"] = np.nan
    psle["Region"] = np.nan

    #Dictionary comprehension that uses regular expression of each Region-District unique code as a key, and sets value to
    #None for now. This was a necessary improvement over the previous Region approach (i.e. assigning 25 variables)
    districts = {re.compile(r'^'+key):None for key in unique_groups['CENTRE CODE']}
    
    #For regions:
    region_groups = unique_groups.groupby('REGION NAME').head(1)
    regions = {re.compile(r'^'+key[:4]):None for key in region_groups['CENTRE CODE']}

    #This has been delayed for a while now, but the groupby function approach preserved indexing prior to groupby, but
    #for our upcoming iterations, it's good to ensure that we reset_index to a 0-based index again.
    unique_groups.reset_index(drop=True, inplace=True)
    region_groups.reset_index(drop=True, inplace=True)

    #Set a counter for iteration, and replace each regular expression key value, with its appropriate location name.
    i=0
    for key,value in districts.items():
        districts[key]=unique_groups.loc[i,'DISTRICT NAME']
        i+=1
    r=0
    for key,value in regions.items():
        regions[key]=region_groups.loc[r,'REGION NAME']
        r+=1
    
    #Finally, apply the new spatial categories to the PSLE data. You'll get settingwithcopywarning.
    for key, value in districts.items():
        psle.loc[:,'District'][psle.loc[:, 'CAND_NO'].str.match(key)] = value
    
    for key, value in regions.items():
        psle.loc[:,'Region'][psle.loc[:, 'CAND_NO'].str.match(key)] = value
    
    return psle

#Run the function
psle2017 = categorize_locations(psle2017,get_unique_groups(excel))

#As always, check to ensure intended effects. And it should now have spatial data updated appropriately.
print(psle2017.head())

/Users/Muse/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/Muse/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Muse/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


         CAND_NO  SEX  Kiswahili  English  Maarifa  Hisabati  Science  \
0  PS0107001-001    0        4.0      1.0      2.0       1.0      3.0   
1  PS0107001-002    0        4.0      2.0      2.0       2.0      2.0   
2  PS0107001-003    0        4.0      2.0      2.0       2.0      2.0   
3  PS0107001-004    0        3.0      2.0      2.0       3.0      4.0   
4  PS0107001-005    0        3.0      2.0      3.0       2.0      3.0   

   Average_Grade  CalcAverage    District  Region  
0            2.0          2.2  NGORONGORO  ARUSHA  
1            2.0          2.4  NGORONGORO  ARUSHA  
2            2.0          2.4  NGORONGORO  ARUSHA  
3            3.0          2.8  NGORONGORO  ARUSHA  
4            3.0          2.6  NGORONGORO  ARUSHA  


In [82]:
#Save complete file to desired location.
#psle2017.to_csv("~/...", index=False)

#Can check for any nan in District column by:
#psle2017.District.unique()

And that's how you reverse engineer regions/districts from Candidate Numbers!